# Assignment 2

- Ankita Kokkera - 06032419
- Aria Wang - 06047688
- Tsamara Esperanti Erwin - 06042275
- Jean-Marc Yao - 06055972
- Amer Mulla - 06027165

Import helpers

In [4]:
import pandas as pd
import numpy as np
from naive_bayes import NaiveBayesForSpam

1. Load the four dataset files into Python data frames and the two list files into lists.

In [5]:
training = pd.read_csv("training.txt")
validation = pd.read_csv("validation.txt")
test1 = pd.read_csv("test1.txt")
test2 = pd.read_csv("test2.txt")

with open("censored_list_test1.txt", "r", encoding="utf-8") as f:
    test1_list = f.read().splitlines()

with open("censored_list_test2.txt", "r", encoding="utf-8") as f:
    test2_list = f.read().splitlines()


2. Pre-process the SMS messages: Remove all punctuation and numbers from the SMS messages, and change all messages to lowercase.

In [6]:
for df in [training, validation, test1, test2]:
    df["sms"] = (
        df["sms"]
          .str.lower()
          .str.replace(r"[0-9]", "", regex=True)
          .str.replace(r"[^\w\s]", "", regex=True)
          .str.replace(r"\s+", " ", regex=True)
          .str.strip()
    )

3. Naive Bayes Classifier from code

4. Explain the code: What is the purpose of each function? What do train and train2
do, and what is the difference between them? Where in the code is Bayes’ Theorem
being applied?

#### train
The train function creates a comprehensive model that calculates likelihoods for every unique word found in the dataset.

#### train2
The train2 function acts as a filter that only retains "spammy" keywords. Specifically, ⁠ train2 ⁠ includes a conditional check that only saves a word if it is at least 20 times more likely to appear in a spam message than in a ham message. 

#### Difference between train and train2
The main difference between train and train2 lies in the feature selection and the resulting size of the model's vocabulary. train ⁠ maintains a full linguistic profile of both categories, including neutral and common words.⁠ train2⁠ results in a much smaller, specialized vocabulary focused on high-impact triggers, whereas⁠ 

#### predict
The predict function applies Bayes’ Theorem under the Naïve Bayes independence assumption to compute posterior probabilities for ham and spam, and assigns the class with the higher posterior probability.

#### score
The score function evaluates the performance of the classifier on the dataset by computing prediction accuracy and the confusion matrix.

#### Bayes Theorem
The bayes theorem is applied in the for loop of predict function.

5. Use your training set to train the classifiers train and train2. Note that the interfaces
of our classifiers require you to pass the ham and spam messages separately.

In [7]:
train_ham = training[training["label"] == "ham"]["sms"].tolist()
train_spam = training[training["label"] == "spam"]["sms"].tolist()

In [8]:
nb_all = NaiveBayesForSpam()
nb_all.train(train_ham, train_spam)

nb_spam = NaiveBayesForSpam()
nb_spam.train2(train_ham, train_spam)

6. Using the validation set, explore how each of the two classifiers performs out of sample.

In [9]:
validation_labels = validation["label"].tolist()
validation_sms = validation["sms"].tolist()

In [10]:
# Evaluate train()
acc_all, conf_all = nb_all.score(validation_sms, validation_labels)
print("Classifier using train():")
print("Accuracy:", acc_all)
print("Confusion matrix:\n", conf_all)

Classifier using train():
Accuracy: 0.955
Confusion matrix:
 [[844.  29.]
 [ 16. 111.]]


In [11]:
# Evaluate train2()
acc_spam, conf_spam = nb_spam.score(validation_sms, validation_labels)
print("Classifier using train2():")
print("Accuracy:", acc_spam)
print("Confusion matrix:\n", conf_spam)

Classifier using train2():
Accuracy: 0.963
Confusion matrix:
 [[856.  33.]
 [  4. 107.]]


7. Why is the train2 classifier faster? Why does it yield a better accuracy both on the training and the validation set?

The ⁠train2 ⁠ classifier is faster because it drastically reduces the dimensionality of the model. In the first version, the algorithm must process every unique word in the entire dataset. In ⁠train2 ⁠, however, the model discards any word that isn't at least 20 times more likely to be spam. This results in a much smaller list of ⁠self.words⁠. 

Train2 has better accuracy because it filters out the noises by Regarding the improved accuracy on both the training. Removal of Overlapping Features: In ⁠ train1 ⁠, common words like "the," "meeting," or "hello" might appear in both ham and spam.⁠ train2 ⁠ ensures that only words with a very high signal-to-noise ratio are used for classification. Then it prevents overfitting to noise by ignoring neutral words that don't have a strong logical connection to spam, the model becomes more "robust." Instead of getting distracted by the specific phrasing of a single email, it focuses on the undeniable markers of spam, which tend to be more consistent across both the training data and new, unseen validation data. It mitigates the independence assumption, Naive Bayes assumes all words are independent, which is rarely true in real language. By narrowing the feature set to only the most significant spam words, the model reduces the impact of this flawed assumption, focusing only on the variables that provide the most reliable evidence.

#### Add feature selection


8. How many false positives (ham messages classified as spam messages) did you get
in your validation set? How would you change the code to reduce false positives at
the expense of possibly having more false negatives (spam messages classified as ham
messages)?

By using train(), the number of false positives in the validation set is 16. By using train2(), the number of false positives in the validation set is 4. 
The train2 classifier produces significantly fewer false positives than train, indicating that its feature selection strategy is more conservative in classifying ham messages as spam. To reduce false positives, we increase the decision threshold required to classify a message as spam. Instead of predicting spam when 𝑃(spam∣𝑋)>0.5, we require P(spam∣X)>τ with τ>0.5 (e.g., 0.8). This makes the classifier more conservative, lowering ham→spam errors at the expense of potentially increasing spam→ham errors.

9. Assuming the missing words are (Xj= xj , . . . , Xk = xk) with k ≤p, how would you
change the formula given as hint to calculate P(Y= Cj |X1 = x1, . . . , Xp = xp) ?

10. Modify the predict function in the code to implement the change in the previous point,
and use it to report accuracies on test1, with both ‘train’ and ‘train2’.

In [12]:
## check predict function

class NaiveBayesForSpamNew:
    
    def train (self, hamMessages, spamMessages):
        self.words = set (' '.join (hamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len (spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len (hamMessages)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) / len (spamMessages)
            self.likelihoods.append ([min (prob1, 0.95), min (prob2, 0.95)])
        self.likelihoods = np.array (self.likelihoods).T
        
    def train2 (self, hamMessages, spamMessages):
        self.words = set (' '.join (hamMessages + spamMessages).split())
        self.priors = np.zeros (2)
        self.priors[0] = float (len (hamMessages)) / (len (hamMessages) + len (spamMessages))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        spamkeywords = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len ([m for m in hamMessages if w in m])) / len (hamMessages)
            prob2 = (1.0 + len ([m for m in spamMessages if w in m])) / len (spamMessages)
            if prob1 * 20 < prob2:
                self.likelihoods.append ([min (prob1, 0.95), min (prob2, 0.95)])
                spamkeywords.append (w)
        self.words = spamkeywords
        self.likelihoods = np.array (self.likelihoods).T

    def predict(self, message, spam_threshold=0.8):
        posteriors = np.copy(self.priors)
        for i, w in enumerate(self.words):
            if w in message.lower():
                posteriors *= self.likelihoods[:, i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors / np.linalg.norm(posteriors, ord=1)

        # more conservative spam decision
        if posteriors[1] >= spam_threshold:
            return ['spam', posteriors[1]]
        return ['ham', posteriors[0]]    

    def score (self, messages, labels):
        confusion = np.zeros(4).reshape (2,2)
        for m, l in zip (messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0,0] += 1
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0,1] += 1
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1,0] += 1
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float (confusion.sum()), confusion

In [13]:
nb_all2 = NaiveBayesForSpamNew()
nb_all2.train(train_ham, train_spam)

nb_spam2 = NaiveBayesForSpamNew()
nb_spam2.train2(train_ham, train_spam)

11. Repeat the previous point using test2. Briefly report your findings.